In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4', temperature=0.1)

### Define Separate Grammatical Error Correction and Feedback Generation Chains:

In [3]:
gec_template = '''Reply with a corrected version of the input sentence with all grammatical and spelling errors fixed. If there are no errors, reply with a copy of the original sentence.

Input sentence: I think smoke should to be ban in all restarants.
Corrected sentence: I think smoking should be banned at all restaurants.

Input sentence: We discussed about the issu.
Corrected sentence: We discussed the issue.

Input sentence: {sentence}
Corrected sentence: '''
gec_prompt = PromptTemplate(template=gec_template, input_variables=['sentence'])
gec_chain = LLMChain(prompt=gec_prompt, llm=llm, output_key='correction')

In [4]:
feedback_template = '''Given a sentence and its correction, give a short feedback comment about the grammatical and spelling errors that were fixed. Your response should be easy to understand. If there are no changes, reply with "Looks good!"

Input sentence: I think smoke should to be ban in all restarants.
Corrected sentence: I think smoking should be banned at all restaurants.
Feedback: If you use the passive voice, you need to use the past participle of the verb.

Input sentence: We discussed about the issu.
Corrected sentence: We discussed the issue.
Feedback: We don't use the preposition "about" with the verb "discuss."

Input sentence: {sentence}
Corrected sentence: {correction}
Feedback: '''
feedback_prompt = PromptTemplate(template=feedback_template, input_variables=['sentence', 'correction'])
feedback_chain = LLMChain(prompt=feedback_prompt, llm=llm, output_key='feedback')

### Test individual chains:

In [5]:
sentence = 'its important of university students to part time job'

gec_response = gec_chain.predict(sentence=sentence)
print(gec_response)

It's important for university students to have a part-time job.


In [6]:
feedback_response = feedback_chain.predict(sentence=sentence, correction=gec_response, output_key='feedback')

print(feedback_response)

Use proper contractions, prepositions, and include necessary articles. Also, use hyphens for compound adjectives.


### Define a chain to combine both:

In [7]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[gec_chain, feedback_chain],
    input_variables=['sentence'],
    output_variables=['feedback'])

In [8]:
feedback_response = overall_chain({'sentence':'its important of university students to part time job'})

for key, value in feedback_response.items():
    print(f'{key.title()}:\n{value}')

Sentence:
its important of university students to part time job
Feedback:
Use the correct form of "it's" and include the preposition "for" and the article "a" in the sentence.


## Demo with Gradio

In [9]:
import gradio as gr
import tqdm as tqdm

In [10]:
def generate_feedback(sentence):
    response = overall_chain({'sentence':sentence})
    return response['feedback']

In [11]:
app_inputs = gr.components.Textbox(lines=2, placeholder='Input sentence')

interface = gr.Interface(fn=generate_feedback, 
                        inputs=app_inputs,
                         outputs='text', 
                        title='Feedback Generator')

interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
